# Helium Network
A [Helium hálózat](https://www.helium.com/) egy elosztott, decentralizált wireless peer-to-peer hozzáférési hálózat. Különlegessége, hogy bűrki csatlakoztathatja eszközeit. Az így nyújtott szolgáltatásért cserébe cripto valutát (HNT) bányászhatnak a rout-erek.
A Helium saját hardware-t is szolgáltat, melyek nagy hatósugarú LongFi routerek.
## Proof of Coverage
A decentralizáltságot Blockchain technológiával érik el. Az egyes node-ok pozícióit és az általuk lefedett terület nagyságát a Helium saját, Proof of Coverage algoritmusa szerint határozzák meg. Minden node-ot próba alá vetnek, mely során kap egy üzenetet, melyet tovább broadcast-ol. A hozzá közel elhelyezkedő node-ok ezt elkapják és feljegyzik  a közös registry-be. A witness-ek ezért HNT-t kapnak.

A kapott jutalmat a hálózat leskálázza aszerint, hogy az adott node mennyire sűrűn lefedett területen helyezkedik el.

## Imports

In [19]:
# API
import requests
import datetime
import string
import random

# Data
import networkx as nx
import pandas as pd
import numpy as np
from geopy import distance
from tqdm import tqdm

# Visualization
from holoviews.util.transform import lon_lat_to_easting_northing
import hvplot.pandas
import panel as pn
import param
#pn.extension(comms='ipywidgets')
#pn.extension(comms='vscode')
pn.extension()
from bokeh.plotting import figure, show
from bokeh.tile_providers import OSM, get_provider
from bokeh.models import ColumnDataSource
from bokeh.models.glyphs import Circle

# Graph Neural Network
import torch
import torch.nn.functional as F
from torch_geometric.nn import AGNNConv, GCNConv
from torch_geometric.data import Data, Batch
from torch_geometric.utils import from_networkx
from networkx import read_gpickle
from torch_geometric.data import DataLoader

ModuleNotFoundError: No module named 'torch_geometric'

## Data load
Az adatok lekérdezéséhez a [Helium API](https://docs.helium.com/api/)-ját használtuk. Segítségével adott városokhoz tartozó routerek és azok szomszédai is lekérdezhetőek, így az adatokból gráf építhető.

### Adatok
Minden node-ra a következő adatokat kérdeztük le:
- reward, reward_scale, status
- longitude, latitude
- witnesses

### API calls



In [4]:
class HeliumAPI:
    def __init__(self, agent_id: str = None):
        self.api_url = 'https://api.helium.io/v1/' #API root
        if agent_id is None:
            agent_id = ''.join(random.choices(string.ascii_letters + string.digits, k = 8))    
        self.agent_id = agent_id #kell egy egyedi azonosító

    def __call__(self, endpoint: str, parameters=None):
        # Egy api kérést megvalósító fgv
        url = self.api_url + endpoint
        r = requests.get(url, headers = {'User-agent': self.agent_id}, params=parameters)
        js = r.json()
        if 'data' in js:
            return js['data']
        else: return []

    def city_hotspots(self, city_id: str):
        # adott városhoz tartozó node-ok
        endpoint = f"cities/{city_id}/hotspots"
        return self(endpoint)

    def hotspot_witnesses(self, hotspot_address: str):
        # node szomszédai
        endpoint = f"hotspots/{hotspot_address}/witnesses"
        return self(endpoint)


    def hotspot_reward(self, hotspot_address: str, n_days: int):
        # node által összegyűjtött jutalmak az elműlt n napban
        start_time = datetime.datetime.isoformat(datetime.datetime.now() - datetime.timedelta(days=n_days))
        endpoint = f"hotspots/{hotspot_address}/rewards/sum?min_time={start_time}"
        rewards = self(endpoint)
        if len(rewards) > 0:
            return rewards['total']
        else:
            return 0


    def hotspot_details(self, hotspot_address: str):
        # node adatai
        endpoint = 'hotspots/' + hotspot_address
        return self(endpoint)

    def city_id(self, city_name:str):
        # városhoz tartozó id. Több esetén a legtöbb node-dal rendelkezőt választjuk.
        cities = self(endpoint='cities', parameters={'search': city_name})
        if len(cities) == 0:
            raise f"{city_name} not found"
        biggest = max(cities, key=lambda city:city['online_count'])
        return biggest['city_id']

### Graph
A gráfépítő függvény, illetve az ezt kiegészítő Pagerank és Betweenness függvények

In [5]:
class CityGraph:
    def __init__(self, city_name, neighbours=False):
        print(f"Initializing {city_name}")
        self.api = HeliumAPI()
        self.city_name = city_name
        self.city_id = self.api.city_id(city_name)
        self.g = None
        self.hotspot_list = self.api.city_hotspots(self.city_id)
        for i in range(len(self.hotspot_list)):
            self.hotspot_list[i]['city'] = city_name
        self.witnesses = {}
        if neighbours:
            self.get_neighbours()
    
    def get_neighbours(self):
        # segítségével a város kibővíthető a node-ok szomszédjaival is.
        addresses = []
        for h in self.hotspot_list:
            addresses.append(h['address'])
        neighbour_adresses = []
        neighbours = []
        pbar = tqdm(addresses)
        pbar.set_description("Expanding to neighbours")
        for address in pbar:
            witnesses = self.api.hotspot_witnesses(address)
            self.witnesses[address] = witnesses
            for witness in witnesses:
                new_adress = witness['address']
                if new_adress in addresses or\
                   new_adress in neighbour_adresses:
                    continue
                witness['city'] = f"{self.city_name} neighbour"
                neighbours.append(witness)
                neighbour_adresses.append(new_adress)
        self.hotspot_list += neighbours
        return neighbours
    
    def generate_graph(self):
        # A node-okat összekapcsoljuk witness-ek alapján.
        g = nx.Graph()

        # minden node-ra egy csomópontot veszünk fel.
        pbar = tqdm(self.hotspot_list)
        pbar.set_description("Constructing nodes")
        for hotspot in pbar:
            hotspot['reward'] = self.api.hotspot_reward(hotspot['address'], 5)
            g.add_node(hotspot['address'], reward=hotspot['reward'],
                       gain=hotspot['gain'], elevation=hotspot['elevation'])

        # A csúcspontokat összekötjuk
        pbar = tqdm(self.hotspot_list)
        addresses = []
        for h in self.hotspot_list:
            addresses.append(h['address'])
        pbar.set_description("Connecting nodes")
        for hotspot in pbar:
            address = hotspot['address']
            if address in self.witnesses:
                witness_list = self.witnesses[address]
            else:
                witness_list = self.api.hotspot_witnesses(address)
            hotspot['num_witnesses'] = len(witness_list)
            g.nodes[address]['num_witnesses'] = hotspot['num_witnesses']
            distances = []
            pos = (hotspot['lat'], hotspot['lng'])
            for witness in witness_list:
                if witness['address'] in addresses:
                    # node-ok méterben mért geodéziai távolsága lesz az élek súlya
                    witness_pos = (witness['lat'], witness['lng'])
                    dist = distance.distance(pos, witness_pos).m
                    distances.append(dist)
                    g.add_edge(address, witness['address'], weight=dist)
            # egy router hatósugarát a hozzátartozó witness-ektől vett távolságok
            # 75. percintilisével becsüljük.
            hotspot['range'] = 0. if len(distances) == 0 else np.percentile(distances, 75)
        self.g = g
        return g

    def pagerank(self):
        # futtatja a pagerank algoritmust a felépített hálóra
        if not self.g:
            raise NameError('You must generate a graph first with CityGraph.generate_graph()')
        print('Pagerank: ', end='')
        pr = nx.pagerank(self.g)
        for hotspot in self.hotspot_list:
            hotspot['pagerank'] = pr[hotspot['address']]
        print('Done')
        return pr

    def betweenness_centrality(self):
        # futtatja a betweenness centrality algoritmust a felépített hálóra
        if not self.g:
            raise NameError('You must generate a graph first with CityGraph.generate_graph()')
        print('Betweenness: ', end='')
        bc = nx.betweenness_centrality(self.g)
        for i in range(len(self.hotspot_list)):
            self.hotspot_list[i]['betweenness_centrality'] = bc[self.hotspot_list[i]['address']]
        print('Done')
        return bc

    def generate_hotspots(self):
        self.generate_graph()
        self.pagerank()
        self.betweenness_centrality()
        return self.hotspot_list

### Save
A felépített gráfokhoz tartozó node-ok adatait csv fájlokba mentjük.

In [ ]:
def save_hotspots(hotspots, file_name):
    fields = ['address', 'lng', 'lat', 'num_witnesses', 'range', 'reward', 'reward_scale', 'elevation', 'gain', 'pagerank', 'betweenness_centrality', 'city', 'status']
    df = pd.DataFrame(hotspots)
    statuses = []
    for status in df['status']:
        statuses.append(status['online'])
    df['status'] = statuses
    df = df[fields]
    df = df.fillna(0.)
    df.to_csv(f"{file_name}.csv")

In [7]:
hotspots = []
for city in ['budapest', 'wien', 'bratislava']:
    graph = CityGraph(city, True)
    new_hotspots = graph.generate_hotspots()
    save_hotspots(new_hotspots, f"helium_{city}_neighbours")
    hotspots += new_hotspots
save_hotspots(hotspots, f"helium_all_neighbours")

## Vizalizáció

### Load
Az adatok betöltése után hozzákapcsoljuk a betanított modell hibáit és átkonvertáljuk a long-lat koordinátákat Universal Transverse Mercator koordinátákba.

In [31]:
df = pd.read_csv('helium_all_neighbours.csv')
eastings, northings = lon_lat_to_easting_northing(df.lng, df.lat)
df['east'] = eastings
df['north'] = northings
params = ['pagerank',
          'betweenness_centrality',
          'reward_scale',
          'elevation',
          'gain',
          'reward',
          'num_witnesses',
          #'error',
          'city',
          ]
real_params = ['pagerank',
               'betweenness_centrality',
               'reward_scale',
               'reward',
               #'error'
              ]

### Városok
A különböző városokba tartozó node-okat különböző színekkel jelenítjük meg.
Megkülönböztetjük az adott városba tartozó és az ő közvetlen szomszédaikból node halmazokat.


Bécs esetében az egyes node-okat egész jól lettek felcímkézve, de például Budapestnél a városon belül található node-ok fele csak a szomszédokba tartozik.


Érdekesség, hogy Pozyon és Bécs összeérnek, de Budapest is produkált kiugró értékeket. (lsd, Balaton alatt.)

In [6]:
df.hvplot.points(x='east', y='north', c='city', alpha=0.6, tiles='OSM', width=750, height=400)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [east,north]   (city)

### Range
A node-ok hatótávolságát a szomszédoktól vett geodéziai távolságok 75. percentilisével közelítettük. Ez a megközelítés azt a hibát eredményezte, hogy kevés szomszéd esetén egészen kiugró értékeket kaptunk.


A Helium hálózat lefedettségére heatmap szerűen egymásra vetítettük a node-ok hatótávolságait. Hogy a távolságok ne kövesség a Bokeh zoom-ját, alacsony szinten kellet megírni a kirajzolást.

In [9]:
p = figure(plot_width = 750, plot_height = 400)
ranges = Circle(x='east', y='north', radius='range', fill_color="blue", fill_alpha=0.08, line_color=None)
centers = Circle(x='east', y='north', radius=100, fill_color="red", fill_alpha=0.5, line_color=None)
data_source = df.copy()
data_source['range'] /= 5
data_source = ColumnDataSource(data_source)
p.add_glyph(data_source, glyph=ranges)
p.add_glyph(data_source, glyph=centers)
p.add_tile(get_provider(OSM))
show(p)

### További értékek
Az adatok átláthatósága kedvéért egy univerzális plot-ot készítettünk. Segítségével lekérdezhető a node-ok tetszőleges adata, akár datashader-rel is.

In [10]:
# A városok csoportosítására használt függvény
combined_cities = ['all', 'wien all', 'budapest all', 'bratislava all']
all_cities = list(df.city.unique())+ combined_cities
def select_city(city: str, df):
    if city == 'all':
        return df
    elif city in list(df.city.unique()):
        return df[df.city == city]
    else:
        return df[df.city.str.contains(city.split()[0], regex=False)]

In [61]:
class ScatterPlot(param.Parameterized):
    value  = param.Selector(objects=params)
    map_type  = param.Selector(objects=[None, 'OSM', 'EsriImagery'])
    city  = param.Selector(objects=all_cities)
    shade = param.Boolean(False)
    #window    = param.Integer(default=10, bounds=(1, 20))
    #sigma     = param.Number(default=10, bounds=(0, 20))
    
    def __init__(self, data=df, *args, **kwargs):
        self.data = data
        super(ScatterPlot, self).__init__(*args, **kwargs)
    
    def view(self):
        __df = select_city(self.city, self.data)
        if self.shade:
            return __df.hvplot.points(x='east',
                                      y='north',
                                      c=self.value,
                                      tiles=self.map_type,
                                      frame_width=500,
                                      frame_height=400,
                                      alpha=0.8,
                                      cmap='Inferno_r',
                                      datashade=self.shade,
                                      #dynamic =False,
                                      dynspread =True,
                                      aggregator ='mean')
        else:
            return __df.hvplot.points(x='east',
                                      y='north',
                                      c=self.value,
                                      tiles=self.map_type,
                                      hover_cols=real_params+['address'],
                                      frame_width=500,
                                      frame_height=400,
                                      s=50,
                                      alpha=0.6,
                                      cmap='Inferno_r')
pl = ScatterPlot(data=df,value='reward_scale', city='wien all', map_type='OSM', shade=True)
pn.Column(pl.param, pl.view)

Column
    [0] Column(margin=5, name='ScatterPlot', width=300)
        [0] StaticText(value='<b>ScatterPlot</b>')
        [1] Select(name='Value', options=OrderedDict([('pagerank', ...]), value='reward_scale')
        [2] Select(name='Map type', options=OrderedDict([('None', ...]), value='OSM')
        [3] Select(name='City', options=OrderedDict([('budapest', ...]), value='wien all')
        [4] Checkbox(name='Shade', value=True)
    [1] ParamMethod(method)

## EDA

In [43]:
df.describe()

,Unnamed: 0,lng,lat,num_witnesses,range,reward,reward_scale,elevation,gain,pagerank,betweenness_centrality,east,north
count,2181.00000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2.181000e+03,2.181000e+03
mean,1090.00000,17.525833,47.936266,48.858780,9139.034719,0.663214,0.484029,14.054104,43.662540,0.001376,0.001158,1.950967e+06,6.096432e+06
std,629.74479,1.228917,0.329284,41.995546,10183.251311,0.822152,0.321188,16.144184,19.811817,0.001554,0.003035,1.368024e+05,5.460750e+04
min,0.00000,16.073297,46.263697,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000178,0.000000,1.789271e+06,5.822708e+06
25%,545.00000,16.380872,47.549111,13.000000,3775.417022,0.095871,0.219620,5.000000,30.000000,0.000399,0.000013,1.823510e+06,6.032168e+06
50%,1090.00000,17.110993,48.144698,42.000000,7190.616850,0.391771,0.366928,10.000000,40.000000,0.000857,0.000146,1.904787e+06,6.130961e+06
75%,1635.00000,19.042719,48.197906,76.000000,11552.821774,0.962839,0.833328,20.000000,58.000000,0.001747,0.000888,2.119826e+06,6.139843e+06
max,2180.00000,20.136369,48.477003,202.000000,147879.050756,9.035526,1.000000,250.000000,150.000000,0.019078,0.044773,2.241570e+06,6.186581e+06


### Hisztogramok elemzése
Az adatok elemzésére a statisztikai egy gyakran használatos ábrázolási módszerét használtuk, történetesen a hisztogramokat. Ezek lehetőséget adnak az adatok gyakoriságának vizsgálatára illetve utalnak az eloszlásra is.

Négy adatot vizsgáltunk igazán. Ezek az adatok a háló építése során is előkerülhetnek, ezért lényeges volt megismernünk az elemeket.

A sűrűségfüggvényeken egyfajta haranggörbe jelleget vélhetünk feltételezni, de ezek a görbék több hullámot tartalmaznak. Ezek nem jellemzik a normális eloszlást, bár a haranggörbe jelleg erre utalna.

Egyedül a ***Pagerank*** értéke mely a haranggörbe jelleget legjobban mutatja, bár ezen értékek egyfajta transzformálása során nem kaptuk meg a várt eredményt. Sem vizuálisan, sem Shapiro teszt segítségével elutasíthatjuk azon hipotézisünket, amely a normális eloszlást feltételezi.

 A több hullám a haranggörbe esetén úgynevezett Körkörös adatsorra enged következtetni melyhez tartozó eloszlás az úgynevezett "*Körkörös eloszlás*". Bár ezt csak vizuálisan tudtuk ellenőrizni, mivel nincs olyasfajta teszt, amellyel ezen feltételezést tudtuk volna tesztelni.

In [70]:
class HistPlot(param.Parameterized):
    value  = param.Selector(objects=real_params)
    city  = param.Selector(objects=all_cities)
    use_kde = param.Boolean(True)
    
    def __init__(self, data=df, *args, **kwargs):
        self.data = data
        super(HistPlot, self).__init__(*args, **kwargs)

    def view(self):
        __df = select_city(self.city, self.data)
        if self.use_kde:
            return __df.hvplot.hist(self.value) * __df.hvplot.kde(self.value)
        else:
            return __df.hvplot.hist(self.value)
p = HistPlot(data=df)
pn.Column(p.param, p.view)

Column
    [0] Column(margin=5, name='HistPlot', width=300)
        [0] StaticText(value='<b>HistPlot</b>')
        [1] Select(name='Value', options=OrderedDict([('pagerank', ...]), value='pagerank')
        [2] Select(name='City', options=OrderedDict([('budapest', ...]), value='budapest')
        [3] Checkbox(name='Use kde', value=True)
    [1] ParamMethod(method)

### Boxplotokkal való vizsgálódás
Boxplotokra azért van szükségünk, hogy az elemekről konkrétabb megfogalmazásokat tudjunk hozni. Fentebb láthatjuk az egész adatsorra vonatkozó értékeket, de fontos hogy az egyes adatokat városra való lebontásban is képesek legyünk elemezni. Ugyancsak három értéket vizsgálunk. Ismétlem, azért korlátozódunk ezekre az értékekre mert a háló építés során ezeket az értékeket használjuk fel.
#### Reward scale
Minden esetben szembetűnő a kiugró adatok sokasága. Látható, hogy Bécs értékei a legjobban elosztottak a (0,1) intervallumon. Mivel Bécs rendelkezik a legtöbb elemszámmal ezért nyilvánvaló, hogy sokkal jobban eloszlanak az értékek, mint Pozsony vagy Budapest esetében. 
Pozsony értékei inkább az 1-es érték közelében helyezkedik el még a Budapesti értékek inkább a 0-ás érték közelében. 
Érdemes megvizsgálni, hogy a node-ok száma, azok közötti távolság mennyiben befolyásolja ezen értékeket.
Emellett látszik, hogy a medián egy olyan érték, mely a vizsgálódás során fontos lehet, a kvartilisakkal egyetemben

#### Betweenness centrality
A node-ok közötti távolság egy értéke mely (0, 0.05) intervallumon mozog. Minél magasabb az érték annál több a kapcsolódó node-ok száma.
Látható hogy az egyes értékek inkább az intervallum alsóbb értékénél tömörülnek illetve, hogy itt is sok kiugró adatot láthatunk. 
Mivel az értékek inkább a kisebb értékek felé tömörülnek, ezért nem állíthatjuk, hogy olyan nagy befolyással lenne a Reward scale értékére.

#### Pagerank
Az egyes oldalak rangja és a szomszédok száma befolyásolja az értékét. 
A dobozok alapján itt is egyből szembetűnik a kiugró adatok sokasága.
Elhelyezkedésük és jellegük alapján egyfajta összefüggést lehet feltételezni a Betweennes centrality értékével. Feltételezzük, hogy erősen korreláltak.
(Evidensnek mondható, hiszen mind a két érték függ a szomszédok számától)

#### Reward
A ***reward*** ábrája hasonló mind a ***betwenness centrality*** mind a ***pagerank*** esetében. Ezzel feltételezhetünk egy összefüggést közöttük. Inkább a nullához közelebbi értékeket veszi fel. Hasonlóan a fentiekhez sok kiugró adatokat kapunk. Emellett a medián értéke itt is egy határt képez az adatok sokasága esetében ez pedig összefüggést jelenthet a ***Reward scale*** értékkel. (Ami ismételten evidens, hiszen az egyik érték a másikból adódik)

In [48]:
class BoxPlot(param.Parameterized):
    value  = param.Selector(objects=real_params, default=real_params[0])
    city  = param.Selector(objects=combined_cities+list(['parallel']))

    def view(self):
        if self.city == 'parallel':
            __df = df.copy()
            for city_name in ['budapest', 'wien', 'bratislava']:
                __df[__df.city.str.contains(city_name)].city = city_name
            return __df.hvplot.box(self.value, by='city')
        else:
            return select_city(self.city, df).hvplot.box(self.value)
p = BoxPlot()
pn.Column(p.param, p.view)

Column
    [0] Column(margin=5, name='BoxPlot', width=300)
        [0] StaticText(value='<b>BoxPlot</b>')
        [1] Select(name='Value', options=OrderedDict([('pagerank', ...]), value='pagerank')
        [2] Select(name='City', options=OrderedDict([('all', ...]), value='all')
    [1] ParamMethod(method)

### Korreláció
Előbb feszegettünk pár összefüggést az értékek között érdemes vizuális ábrázolás mellett következtetéseket levonni.

#### Reward és Betweeness centrality
A ***reward*** értéke és a ***betweenness centrality*** erősen korreláltak.
Ez arra enged következtetni, hogy nagyobb nyeremény jár azoknak a bányászoknak akik közel vannak egymáshoz. Ezzel is utalva arra hogy a Helium maga a "csapatmunkát" díjazza.

#### Pagerank és Betweenness-centrality
A két érték közötti korreláció erősnek módható, ahogy ezt fent már feltételeztük.Ez arra adhat következtetést hogy az egyes node-ok pagerank-je inkább a szomszédok száma függvényében van pontozva, így evidens hogy a ***betweenness centrality *** értékével erősen összefügg.

#### Reward scale és reward.
Nem meglepő hogy a két érték között valamilyen korreláció tapasztalható. Egyik a másikból adódik.

#### Pagerank és Reward scale.
Bár gyengén korrelálnak ez annak köszönhető hogy az egyes ***reward scale*** értékek mind a szomszédok számával, mind a ***reward*** értékével és a jellenlegi státusszal leírható érték. Mivel a Pagerank a szomszédok számával szoros összefüggésben van, ezért evidens, hogy a két érték is valamilyen korreláltságot mutat

#### Nincs korreláció
Ahogy a boxplotok alapján feltételeztük a ***reward scale*** és a ***betweenness centrality*** között nincs korreláció. Míg a ***reward*** értéke függ attól hogy milyen közel helyezkedünk el egymáshoz, a ***reward scale*** esetén ez irreleváns információ.

Bár nem vizsgáltuk mégis többször szóbakerült, esetleges függés van a ***number of witnesses*** és a ***betweenness centrality*** vagy a ***reward scale*** között. Vizuálizációból az látszik, hogy az érték között nincs összefüggés.

In [50]:
class BivariatePlot(param.Parameterized):
    x  = param.Selector(objects=real_params, default=real_params[0])
    y  = param.Selector(objects=real_params, default=real_params[1])
    plot_type  = param.Selector(objects=['bivariate', 'scatter'])
    city  = param.Selector(objects=all_cities)

    def view(self):
        __df = select_city(self.city, df)
        if self.plot_type == 'bivariate':
            plot = __df.hvplot.bivariate(x=self.x, y=self.y, frame_width=400,
                                                   frame_height=400,
                                                   filled=True)
        else:
            plot = __df.hvplot.scatter(x=self.x, y=self.y, width=400, c='blue', alpha=0.5)
        return plot
p = BivariatePlot()
pn.Column(p.param, p.view)

Column
    [0] Column(margin=5, name='BivariatePlot', width=300)
        [0] StaticText(value='<b>BivariatePlot</b>')
        [1] Select(name='X', options=OrderedDict([('pagerank', ...]), value='pagerank')
        [2] Select(name='Y', options=OrderedDict([('pagerank', ...]), value='betweenness_centrality')
        [3] Select(name='Plot type', options=OrderedDict([('bivariate',...]), value='bivariate')
        [4] Select(name='City', options=OrderedDict([('budapest', ...]), value='budapest')
    [1] ParamMethod(method)

## Graph Neural Network
A következőkben a 3 vizsgált városra legeneráljuk a networkx gráfokat, ami a GNN tanítás bemenete lesz. A célváltozó a jutalom lesz, amire majd predikciókat teszünk, azt vizsgálva, hogy van-e szignifikáns eltérés valamelyik hotspotnál a valódi reward és a rendelkezésre álló adatok alapján "jogosnak" vélt reward között.

Ha ilyet találunk arról a hotspotról feltételezhető, hogy valamilyen módon átveri a rendszert, vagy esetleg kevesebb rewardot kap, mint ami megilletné.

### Pozsony

Az első példatelepülés Pozsony, de mindhárom városra ugyanúgy megy a tanítás és anomáliadetektálás folyamata. Először legeneráljuk a gráfot, a csúcsokba pedig elmentjük a később szükséges tulajdonságokat (elevation, Pagerank, stb).

In [11]:
pozsony = CityGraph('bratislava')

Initializing bratislava


In [12]:
pozsony_graph = pozsony.generate_graph()

Connecting nodes: 100%|██████████| 147/147 [03:48<00:00,  1.56s/it]


In [13]:
bc = nx.betweenness_centrality(pozsony_graph)
nx.set_node_attributes(pozsony_graph, bc, "betweenness")
pg = nx.pagerank(pozsony_graph)
nx.set_node_attributes(pozsony_graph, pg, "pagerank")

Alább látható egy csúcs a tulajdonságaival együtt. Kicsi a gráf és kevés a kapcsolat közöttük, valószínűleg ennek tudhatók be az alacsony Pagerank és Betweenness értékek.

In [70]:
list(pozsony_graph.nodes(data=True))[0]

('11BeobVV8SENABJ8jPcsnYofDkpTRvkSjJV5pUsDawX3Qytx7S2',
 {'betweenness': 0.00021620279108006848,
  'elevation': 15,
  'gain': 30,
  'num_witnesses': 37,
  'pagerank': 0.002701869330742776,
  'reward': 0.22128482})

In [15]:
data_full_pozsony = from_networkx(pozsony_graph)

A bemeneti változók az elevation (milyen magasan van az adott hotspot), num_witnesses, azaz a szomszédok száma, illetve a Pagerank és Betweenness Centrality mutatók. A célváltozó a korábban említett reward. Fontos még, hogy a modell felhasználja az élsúlyokat is, ami nem más, mint a két csúcs (hotspot) közti távolság. \\
A kevés adatpontra való figyelemmel, és mivel szeretnénk az összes hotspot körében anomáliákat keresni, maszkolást alkalmazunk, hogy valamennyire eltorzítsuk az eredeti adatpontokat. Így a predikciónál nem egy az egyben ugyanazokat a vektorokat látja a modell, mint a tanításnál. 

In [17]:
features = torch.hstack((data_full_pozsony.elevation.reshape((data_full_pozsony.num_nodes,1)), data_full_pozsony.num_witnesses.reshape((data_full_pozsony.num_nodes,1)),
                         data_full_pozsony.betweenness.reshape((data_full_pozsony.num_nodes,1)),
                         data_full_pozsony.pagerank.reshape((data_full_pozsony.num_nodes,1)))) # változók kiválasztása

# tanító, validációs és teszt adathalmazok előkészítése:

mask = np.zeros((data_full_pozsony.num_nodes))
train_mask = mask.astype(bool)
train_mask[:int(data_full_pozsony.num_nodes*0.7)] = True

val_mask = mask.astype(bool)
val_mask[int(data_full_pozsony.num_nodes*0.7)+1:int(data_full_pozsony.num_nodes*0.9)] = True

test_mask = mask.astype(bool)
test_mask[int(data_full_pozsony.num_nodes*0.9)+1:] = True

data = Data(x=features.float(),
            y=data_full_pozsony.reward.float(),
            edge_index=data_full_pozsony.edge_index,
            train_mask=torch.Tensor(train_mask).type(torch.bool),
            val_mask=torch.Tensor(val_mask).type(torch.bool),
            test_mask=torch.Tensor(test_mask).type(torch.bool),
            pos=data_full_pozsony.pos,
            )

In [18]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = torch.nn.Linear(data.num_features, 16)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)
        self.conv1 = GCNConv(data.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, 1, cached=True)
        self.lin2 = torch.nn.Linear(16, 16)
        self.lin3 = torch.nn.Linear(16, 1)

    def forward(self):

        x, edge_index, edge_weight, pos = data.x, data.edge_index, data.edge_attr, data.pos
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = self.conv2(x, edge_index, edge_weight)
        x = F.dropout(x, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_pozsony, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model_pozsony.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model_pozsony.train()
    optimizer.zero_grad()
    F.mse_loss(model_pozsony()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model_pozsony.eval()
    pred, losses = model_pozsony(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):

        loss = F.mse_loss(data.y[mask], pred[mask])

        losses.append(loss)
    return losses


best_val_loss = test_loss = 0
for epoch in range(1, 101):
    train()
    train_loss, val_loss, test_loss = test()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_loss, val_loss, test_loss))


pred_pozsony = model_pozsony().detach().numpy() #predikciók
real_pozsony = data.y.detach().numpy() #eredeti rewardok

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Using a target size (torch.Size([102])) that is different to the input size (torch.Size([102, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([102, 1])) that is different to the input size (torch.Size([102])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([29, 1])) that is different to the input size (torch.Size([29])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([14, 1])) that is different to the input size (to

Epoch: 001, Train: 7.6647, Val: 8.8200, Test: 10.2571
Epoch: 002, Train: 4.0353, Val: 9.2960, Test: 3.2603
Epoch: 003, Train: 6.9698, Val: 14.5151, Test: 5.8931
Epoch: 004, Train: 9.8069, Val: 17.6479, Test: 9.4713
Epoch: 005, Train: 10.3141, Val: 17.7011, Test: 10.3771
Epoch: 006, Train: 8.4197, Val: 14.8493, Test: 8.3655
Epoch: 007, Train: 5.6610, Val: 10.8950, Test: 5.3217
Epoch: 008, Train: 3.2247, Val: 7.0932, Test: 2.7576
Epoch: 009, Train: 1.9537, Val: 4.4018, Test: 1.7392
Epoch: 010, Train: 1.9957, Val: 3.1572, Test: 2.3655
Epoch: 011, Train: 2.7296, Val: 2.9561, Test: 3.7253
Epoch: 012, Train: 3.4564, Val: 3.1532, Test: 4.8793
Epoch: 013, Train: 3.4381, Val: 3.1526, Test: 4.7807
Epoch: 014, Train: 2.8292, Val: 2.9328, Test: 3.7249
Epoch: 015, Train: 2.0894, Val: 2.7663, Test: 2.4272
Epoch: 016, Train: 1.6218, Val: 2.9262, Test: 1.4852
Epoch: 017, Train: 1.6894, Val: 3.5623, Test: 1.2871
Epoch: 018, Train: 2.2545, Val: 4.5561, Test: 1.8211
Epoch: 019, Train: 2.9970, Val: 5.5504

A predikciók vektora:

In [71]:
pred_pozsony_2 = np.hstack(pred_pozsony)

Az eltéréseket szótárakba rendezzük, hogy visszakereshetőek legyenek az egyes csúcsok ezekkel az indexekkel.

In [21]:
dict_pozsony = {}
for i in range(len(real_pozsony)):
    dict_pozsony[i] = abs(real_pozsony-pred_pozsony_2)[i]

In [72]:
elojeles_hiba_pozsony = pred_pozsony_2-real_pozsony
addressek_pozsony = []
for i in range(len(real_pozsony)):
    addressek_pozsony.append(list(pozsony_graph.nodes(data=True))[i][0])
hiba_df_pozsony = pd.DataFrame(list(zip(addressek_pozsony, elojeles_hiba_pozsony)),
               columns =['address', 'error'])

In [83]:
hiba_df_pozsony.to_csv('pozsony_hiba.csv')

In [26]:
hiba_df_pozsony.sort_values(by=['error'])

,Address,Hiba
116,112o1vnXH7qzfDtsuLdRbRWjnVVTGNz5iZt69e9izAUjVB...,-3.611428
75,1125yWHmsLM1ETCDBh3fmfVRbs6L97f7WNXYSr2x1wpxZz...,-2.884251
130,112YQmnH9rcy4EJKt4ueWRL1FTEooUWd8y2anNKd7eKQ3G...,-2.826682
41,112NfbA2k2PV4cmEu2ZWhrXvDWEWjUYmtTs1DLLxdGmC1G...,-1.873737
72,112JtRLLk42xCEbiacyrk8B5zY8qvtVBxkFBKVYaYLA9Ae...,-1.562317
...,...,...
19,11BLp7Df8Gj5Pzj1EybXMsJ1Smw9KsF9fjEgwm9Fsso6HZ...,0.382669
25,1124ZcC3G1tKcnTuauPpe3554QCBhtZKFhhckCybpxJFmA...,0.391277
82,112brkfjVtJ1BUfWwbyeWiFdrfWw7DXcM3KUo1mNhdHTPt...,0.416629
8,112DopRCs3f1qJ4Nb7VMF3Hr3zZ85T7fBpedPEdcWg3E7e...,0.454939


A fenti táblázatban láthatók a legnagyobbat hibázó hotspotok címei, ahol a legnegatívabb a hiba, ott kap a valóságban jóval több rewardot a hotspot, mint amennyit a modell szerint érdemelne, csalás szempontjából ezek a legérdekesebb esetek.

In [ ]:
sorted_values = sorted(dict_pozsony.values())
sorted_dict = {}

for i in sorted_values:
    for k in dict_pozsony.keys():
        if dict_pozsony[k] == i:
            sorted_dict[k] = dict_pozsony[k]
            break

print(sorted_dict[:3])

A három legnagyobb abszolút eltérést a fenti indexek hozták, a Helium Explorerben megtekinthetők ezek a konkrét felhasználók. 

In [73]:
print('Pozsony esetében a legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/' + hiba_df_pozsony.sort_values(by=['Hiba'])['Address'].iloc[0])

Pozsony esetében a legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/112o1vnXH7qzfDtsuLdRbRWjnVVTGNz5iZt69e9izAUjVBh2qe1A


Látszólag azokat a hotspotokat értékeli félre a modell, amelyek nagyon távoli tanúkkal is rendelkeznek (ez bezavarhat az élsúlyoknak és felboríthatja a modell súlyait), maguk pedig aránylag sűrűn lefedett területen vannak.

A többi város esetében is hasonlóképpen járunk el.

### Bécs

In [27]:
becs = CityGraph('wien')

Initializing wien


In [28]:
becs_graph = becs.generate_graph()

Connecting nodes: 100%|██████████| 788/788 [21:04<00:00,  1.60s/it]


In [29]:
bc = nx.betweenness_centrality(becs_graph)
nx.set_node_attributes(becs_graph, bc, "betweenness")
pg = nx.pagerank(becs_graph)
nx.set_node_attributes(becs_graph, pg, "pagerank")

In [74]:
list(becs_graph.nodes(data=True))[0]

('112Kn3MBsC8fuqqcaxuEwYcP7d5XAu5wxozE3k4MskZjRnEk9mbP',
 {'betweenness': 0.006757136568175682,
  'elevation': 140,
  'gain': 19,
  'num_witnesses': 36,
  'pagerank': 0.00397791202235623,
  'reward': 0.537816})

In [31]:
data_full_becs = from_networkx(becs_graph)

In [32]:
features = torch.hstack((data_full_becs.elevation.reshape((data_full_becs.num_nodes,1)), data_full_becs.num_witnesses.reshape((data_full_becs.num_nodes,1)),
                         data_full_becs.betweenness.reshape((data_full_becs.num_nodes,1)),
                         data_full_becs.pagerank.reshape((data_full_becs.num_nodes,1))))

mask = np.zeros((data_full_becs.num_nodes))
train_mask = mask.astype(bool)
train_mask[:int(data_full_becs.num_nodes*0.7)] = True

val_mask = mask.astype(bool)
val_mask[int(data_full_becs.num_nodes*0.7)+1:int(data_full_becs.num_nodes*0.9)] = True

test_mask = mask.astype(bool)
test_mask[int(data_full_becs.num_nodes*0.9)+1:] = True

data_becs = Data(x=features.float(),
            y=data_full_becs.reward.float(),
            edge_index=data_full_becs.edge_index,
            train_mask=torch.Tensor(train_mask).type(torch.bool),
            val_mask=torch.Tensor(val_mask).type(torch.bool),
            test_mask=torch.Tensor(test_mask).type(torch.bool),
            pos=data_full_becs.pos,
            )

In [33]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = torch.nn.Linear(data_becs.num_features, 16)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)
        self.conv1 = GCNConv(data_becs.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, 1, cached=True)
        self.lin2 = torch.nn.Linear(16, 16)
        self.lin3 = torch.nn.Linear(16, 1)

    def forward(self):

        x, edge_index, edge_weight, pos = data_becs.x, data_becs.edge_index, data_becs.edge_attr, data_becs.pos
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = self.conv2(x, edge_index, edge_weight)
        x = F.dropout(x, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_becs, data_becs = Net().to(device), data_becs.to(device)
optimizer = torch.optim.Adam(model_becs.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model_becs.train()
    optimizer.zero_grad()
    F.mse_loss(model_becs()[data_becs.train_mask], data_becs.y[data_becs.train_mask]).backward()
    optimizer.step()


def test():
    model_becs.eval()
    pred, losses = model_becs(), []
    for _, mask in data_becs('train_mask', 'val_mask', 'test_mask'):

        loss = F.mse_loss(data_becs.y[mask], pred[mask])

        losses.append(loss)
    return losses


best_val_loss = test_loss = 0
for epoch in range(1, 101):
    train()
    train_loss, val_loss, test_loss = test()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_loss, val_loss, test_loss))


pred_becs = model_becs().detach().numpy()
real_becs = data_becs.y.detach().numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Using a target size (torch.Size([551])) that is different to the input size (torch.Size([551, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([551, 1])) that is different to the input size (torch.Size([551])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([157, 1])) that is different to the input size (torch.Size([157])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([78, 1])) that is different to the input size (

Epoch: 001, Train: 2.3778, Val: 2.4301, Test: 3.0229
Epoch: 002, Train: 5.6771, Val: 5.9975, Test: 6.9366
Epoch: 003, Train: 2.4579, Val: 2.5161, Test: 3.1068
Epoch: 004, Train: 0.5927, Val: 0.4814, Test: 0.7764
Epoch: 005, Train: 1.5849, Val: 1.5252, Test: 1.7692
Epoch: 006, Train: 2.3994, Val: 2.3932, Test: 2.6554
Epoch: 007, Train: 1.3125, Val: 1.2352, Test: 1.4756
Epoch: 008, Train: 0.5917, Val: 0.4784, Test: 0.7637
Epoch: 009, Train: 1.2445, Val: 1.1978, Test: 1.6266
Epoch: 010, Train: 2.2851, Val: 2.3272, Test: 2.8917
Epoch: 011, Train: 2.3878, Val: 2.4386, Test: 3.0185
Epoch: 012, Train: 1.4494, Val: 1.4202, Test: 1.8850
Epoch: 013, Train: 0.6717, Val: 0.5689, Test: 0.9022
Epoch: 014, Train: 0.6808, Val: 0.5629, Test: 0.8219
Epoch: 015, Train: 1.0580, Val: 0.9589, Test: 1.1979
Epoch: 016, Train: 1.0809, Val: 0.9829, Test: 1.2208
Epoch: 017, Train: 0.7324, Val: 0.6160, Test: 0.8676
Epoch: 018, Train: 0.5895, Val: 0.4742, Test: 0.7756
Epoch: 019, Train: 0.9010, Val: 0.8206, Test: 

Látható, hogy Bécsre sokkal jobban működött a modell, hiszen az egyes node-ok jobban lettek felcímkézve.

In [75]:
pred_becs_2 = np.hstack(pred_becs)

In [36]:
dict_becs = {}
for i in range(len(real_becs)):
    dict_becs[i] = abs(real_becs-pred_becs_2)[i]

In [76]:
elojeles_hiba_becs = pred_becs_2-real_becs
addressek_becs = []
for i in range(len(real_becs)):
    addressek_becs.append(list(becs_graph.nodes(data=True))[i][0])
hiba_df_becs = pd.DataFrame(list(zip(addressek_becs, elojeles_hiba_becs)),
               columns =['address', 'error'])

In [81]:
hiba_df_becs.to_csv('becs_hiba.csv')

In [ ]:
#hiba_df_becs.sort_values(by=['Hiba'])

In [ ]:
sorted_values = sorted(dict_becs.values())
sorted_dict = {}

for i in sorted_values:
    for k in dict_becs.keys():
        if dict_becs[k] == i:
            sorted_dict[k] = dict_becs[k]
            break

print(sorted_dict[:3])

Ismét sűrűn fedett területekről kikerülő pontok távoli tanúkkal.

### Budapest

In [44]:
budapest = CityGraph('budapest')

Initializing budapest


In [45]:
budapest_graph = budapest.generate_graph()

Connecting nodes: 100%|██████████| 324/324 [08:33<00:00,  1.59s/it]


In [46]:
bc = nx.betweenness_centrality(budapest_graph)
nx.set_node_attributes(budapest_graph, bc, "betweenness")
pg = nx.pagerank(budapest_graph)
nx.set_node_attributes(budapest_graph, pg, "pagerank")

In [78]:
list(budapest_graph.nodes(data=True))[0]

('112t1fBibjhmFF2wUC68FCpMaw47J6UM6awDn7vWacYDhmVdc1kT',
 {'betweenness': 0.0,
  'elevation': 0,
  'gain': 12,
  'num_witnesses': 0,
  'pagerank': 0.0005053908355832823,
  'reward': 0.0})

In [48]:
data_full_budapest = from_networkx(budapest_graph)

In [49]:
features = torch.hstack((data_full_budapest.elevation.reshape((data_full_budapest.num_nodes,1)), data_full_budapest.num_witnesses.reshape((data_full_budapest.num_nodes,1)),
                         data_full_budapest.betweenness.reshape((data_full_budapest.num_nodes,1)),
                         data_full_budapest.pagerank.reshape((data_full_budapest.num_nodes,1))))

mask = np.zeros((data_full_budapest.num_nodes))
train_mask = mask.astype(bool)
train_mask[:int(data_full_budapest.num_nodes*0.7)] = True

val_mask = mask.astype(bool)
val_mask[int(data_full_budapest.num_nodes*0.7)+1:int(data_full_budapest.num_nodes*0.9)] = True

test_mask = mask.astype(bool)
test_mask[int(data_full_budapest.num_nodes*0.9)+1:] = True

data_budapest = Data(x=features.float(),
            y=data_full_budapest.reward.float(),
            edge_index=data_full_budapest.edge_index,
            train_mask=torch.Tensor(train_mask).type(torch.bool),
            val_mask=torch.Tensor(val_mask).type(torch.bool),
            test_mask=torch.Tensor(test_mask).type(torch.bool),
            pos=data_full_budapest.pos,
            )

In [50]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = torch.nn.Linear(data_budapest.num_features, 16)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)
        self.conv1 = GCNConv(data_budapest.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, 1, cached=True)
        self.lin2 = torch.nn.Linear(16, 16)
        self.lin3 = torch.nn.Linear(16, 1)

    def forward(self):

        x, edge_index, edge_weight, pos = data_budapest.x, data_budapest.edge_index, data_budapest.edge_attr, data_budapest.pos
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = self.conv2(x, edge_index, edge_weight)
        x = F.dropout(x, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_budapest, data_budapest = Net().to(device), data_budapest.to(device)
optimizer = torch.optim.Adam(model_budapest.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model_budapest.train()
    optimizer.zero_grad()
    F.mse_loss(model_budapest()[data_budapest.train_mask], data_budapest.y[data_budapest.train_mask]).backward()
    optimizer.step()


def test():
    model_budapest.eval()
    pred, losses = model_budapest(), []
    for _, mask in data_budapest('train_mask', 'val_mask', 'test_mask'):

        loss = F.mse_loss(data_budapest.y[mask], pred[mask])

        losses.append(loss)
    return losses


best_val_loss = test_loss = 0
for epoch in range(1, 101):
    train()
    train_loss, val_loss, test_loss = test()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_loss, val_loss, test_loss))


pred_budapest = model_budapest().detach().numpy()
real_budapest = data_budapest.y.detach().numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Using a target size (torch.Size([226])) that is different to the input size (torch.Size([226, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([226, 1])) that is different to the input size (torch.Size([226])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (to

Epoch: 001, Train: 190.4872, Val: 213.8940, Test: 214.3922
Epoch: 002, Train: 98.3565, Val: 110.3865, Test: 110.5704
Epoch: 003, Train: 38.5311, Val: 43.2200, Test: 43.3216
Epoch: 004, Train: 7.9412, Val: 8.9084, Test: 9.1170
Epoch: 005, Train: 0.4826, Val: 0.6075, Test: 1.0660
Epoch: 006, Train: 9.8638, Val: 11.2328, Test: 12.0196
Epoch: 007, Train: 27.0337, Val: 30.6002, Test: 31.7110
Epoch: 008, Train: 44.3515, Val: 50.1188, Test: 51.4957
Epoch: 009, Train: 55.6637, Val: 62.8600, Test: 64.3999
Epoch: 010, Train: 57.4729, Val: 64.8937, Test: 66.4624
Epoch: 011, Train: 53.0202, Val: 59.8727, Test: 61.3845
Epoch: 012, Train: 42.9570, Val: 48.5310, Test: 49.9018
Epoch: 013, Train: 31.1415, Val: 35.2139, Test: 36.4044
Epoch: 014, Train: 19.6337, Val: 22.2401, Test: 23.2323
Epoch: 015, Train: 10.1422, Val: 11.5342, Test: 12.3291
Epoch: 016, Train: 3.7149, Val: 4.2766, Test: 4.8907
Epoch: 017, Train: 0.6697, Val: 0.8237, Test: 1.2859
Epoch: 018, Train: 0.6846, Val: 0.8147, Test: 1.1577
Epo

In [69]:
pred_budapest_2 = np.hstack(pred_budapest)

In [53]:
dict_bp = {}
for i in range(len(real_budapest)):
    dict_bp[i] = abs(real_budapest-pred_budapest_2)[i]

In [79]:
elojeles_hiba_budapest = pred_budapest_2-real_budapest
addressek_budapest = []
for i in range(len(real_budapest)):
    addressek_budapest.append(list(budapest_graph.nodes(data=True))[i][0])
hiba_df_budapest = pd.DataFrame(list(zip(addressek_budapest, elojeles_hiba_budapest)),
               columns =['address', 'error'])

In [82]:
hiba_df_budapest.to_csv('budapest_hiba.csv')

In [56]:
hiba_df_budapest.sort_values(by=['error'])

,Address,Hiba
300,11k1bBWkD37QnBv6U1ktBkBTJWGbk3p4FKBtw2uNohQRbN...,-3.973993
120,115KUF2f7Ju81vTRJi5mRhj7Joie9CQiznvhceL6cywbKY...,-2.303553
236,118SqUBmzpg7v5aAUAUEhPrJrGPjUYm2CnWrEtkKLSjxZF...,-2.002073
210,11EpA7wDD5LbcfU6WEJRGKXMR8F3LuaAGVP1mBPs678CfF...,-1.929112
237,112eCvWr39eZLykrngENY8FPya5gSjbcRC9qZZnhf3VYuD...,-1.904839
...,...,...
66,112uRNhhLbrYoB7FDrnwx8QhBB5LCVDqk5ejJyr3GkkY2Z...,0.519775
9,112i1LM69hPLcA4FHNbRAzjjmBmK1v58T5QM8rHewxFxsE...,0.568595
277,11q6AMkMpspux5f774L9r9i75po6szfyfyyrzy4Z7aFj1W...,0.590041
7,11LRxWG13jj1xvWQhGMC1YkJu457BqcHyZb2WyHpKc6Sq6...,0.660307


In [ ]:
sorted_values = sorted(dict_bp.values()) 
sorted_dict = {}

for i in sorted_values:
    for k in dict_bp.keys():
        if dict_bp[k] == i:
            sorted_dict[k] = dict_bp[k]
            break

print(sorted_dict[:3])

In [80]:
print('Budapest esetében a legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/' + hiba_df_budapest.sort_values(by=['Hiba'])['Address'].iloc[0])

Budapest esetében a legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/11k1bBWkD37QnBv6U1ktBkBTJWGbk3p4FKBtw2uNohQRbNGDqFD


Budapest esetén is többségében olyan hotspotok szerepelnek, amelyek sűrűn helyezkednek el, és van néhány távoli tanújuk, amik kizökkentik a modellt.

Összességében nehéz bármi mást feltenni ezekről a kapott hotspotokról és felhasználókról, minden bizonnyal egy több várost feldolgozó tanítás után már kaphatnánk egy olyan modellt, ami tud okosabbat mondani és ránézésre is érthető eredményt adni.

### Kitekintés, megjegyzések

Ehhez a GNN-es részhez fontos megjegyzés, hogy csak a maszkoló eljárás miatt tudtuk felhasználni ugyanazt az adathalmazt tanításra és kiértékelésre. Viszont még így is jelen van egy komoly szintű information leakage. Ezen felül a kevés adatpont is problémát okoz, ami az egyes városoknál rendelkezésünkre állt. \\
Ezeknek az orvosolására megpróbáltunk építeni egy olyan modellt is, ami 3 másik nagyobb településen tanult be (Krakkó, Berlin, Zürich), azonban itt az idő szűkössége miatt nem jutottunk el prezentálható eredményig. Ha a jövőben mi vagy valaki más folytatná ezt a projektet, érdekes lehet jobban utánanézni ennek a megoldásnak. 

## Értékelés
A modell predikcióinak hibáit összekapcsoljuk az eredeti adathalmazzal.

In [68]:
error_files = ['becs_hiba.csv', 'pozsony_hiba.csv', 'budapest_hiba.csv']
errors = pd.concat([pd.read_csv('becs_hiba.csv', index_col=0),
                    pd.read_csv('pozsony_hiba.csv', index_col=0),
                    pd.read_csv('budapest_hiba.csv', index_col=0)])
df_merged = pd.merge(df, errors, on='address')
params += ['error']
real_params += ['error']
df_merged.error.describe()

count    1272.000000
mean       -0.304446
std         0.697187
min        -7.616868
25%        -0.459160
50%        -0.080901
75%         0.073482
max         0.730573
Name: error, dtype: float64

Látható, hogy csak a pontosan felcímkézett node-okra prediktáltunk, a szomszédokra nem. Bécs és Pozsony szomszédaiból kiolvasható az átlapolódás, vagyis, hogy egyes node-ok szomszédjai átlógnak a másik városba is.

In [33]:
for city in df.city.unique():
    print(city, df[df.city == city].count()[0], df_merged[df_merged.city == city].count()[0])

budapest 324 324
budapest neighbour 473 0
wien 788 788
wien neighbour 165 4
bratislava 147 147
bratislava neighbour 284 9


In [59]:
pl = ScatterPlot(data=df_merged, value='error', city='wien all', map_type='EsrilImagery', shade=False)
pn.Column(pl.param, pl.view)

Column
    [0] Column(margin=5, name='ScatterPlot', width=300)
        [0] StaticText(value='<b>ScatterPlot</b>')
        [1] Select(name='Value', options=OrderedDict([('pagerank', ...]), value='pagerank')
        [2] Select(name='Map type', options=OrderedDict([('None', ...]))
        [3] Select(name='City', options=OrderedDict([('budapest', ...]), value='budapest')
        [4] Checkbox(name='Shade')
    [1] ParamMethod(method)

Megjelenítve a hibákat, Bécsben látjuk a leginkább kiugró értéket. Jobban megvizsgálva az adott node-ot, látható, hogy valóban átlóg Pozsonyba, így az eredeti adathalmazunkban kétszer is szerepel, egyszer bécsi node-ként, egyszer pedig egy pozsonyi szomszédjaként.

In [42]:
print('A legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/' + df_merged[df_merged.error == df_merged.error.min()].address.values[0])

A legnagyobb hibát produkáló felhasználó a Helium Explorerben: https://explorer.helium.com/hotspots/1125H9FkaAG7tRs53wUUpJSpHFTXTfADuzR2PJPys8WiXYRdRVEM


A node-ot akár a Helium Explorerben is megtekinthetjük.

In [71]:
p = HistPlot(data=df_merged, value='error')
pn.Column(p.param, p.view)

Column
    [0] Column(margin=5, name='HistPlot', width=300)
        [0] StaticText(value='<b>HistPlot</b>')
        [1] Select(name='Value', options=OrderedDict([('pagerank', ...]), value='error')
        [2] Select(name='City', options=OrderedDict([('budapest', ...]), value='budapest')
        [3] Checkbox(name='Use kde', value=True)
    [1] ParamMethod(method)

A histogramról jól leolvasható, hogy többnyire csak negatív irányba tévedett a hálónk. Ez a viselkedés összhangban van azzal a feltevéssel, hogy az outlier-ek rosszándékú tulajdonosok csalási kísérleteinek eredménye és senki sem csalna neki kedvezőtlen irányba.